# Dask for Machine Learning

This is a high-level overview demonstrating some the components of Dask-ML.
Visit the main [Dask-ML](http://ml.dask.org) documentation, see the [dask tutorial](https://github.com/dask/dask-tutorial) notebook 08, or explore some of the other machine-learning examples.

In [1]:
# from dask.distributed import Client, progress
# client = Client(processes=False, threads_per_worker=4,
#                 n_workers=1, memory_limit='2GB')
# clients

from tools import init_cluster

### Create Scikit-Learn Estimator

In [2]:
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd

We'll use scikit-learn to create a pair of small random arrays, one for the features `X`, and one for the target `y`.

In [3]:
X, y = make_classification(n_samples=1000, random_state=0)
X[:5]

array([[-1.06377997,  0.67640868,  1.06935647, -0.21758002,  0.46021477,
        -0.39916689, -0.07918751,  1.20938491, -0.78531472, -0.17218611,
        -1.08535744, -0.99311895,  0.30693511,  0.06405769, -1.0542328 ,
        -0.52749607, -0.0741832 , -0.35562842,  1.05721416, -0.90259159],
       [ 0.0708476 , -1.69528125,  2.44944917, -0.5304942 , -0.93296221,
         2.86520354,  2.43572851, -1.61850016,  1.30071691,  0.34840246,
         0.54493439,  0.22532411,  0.60556322, -0.19210097, -0.06802699,
         0.9716812 , -1.79204799,  0.01708348, -0.37566904, -0.62323644],
       [ 0.94028404, -0.49214582,  0.67795602, -0.22775445,  1.40175261,
         1.23165333, -0.77746425,  0.01561602,  1.33171299,  1.08477266,
        -0.97805157, -0.05012039,  0.94838552, -0.17342825, -0.47767184,
         0.76089649,  1.00115812, -0.06946407,  1.35904607, -1.18958963],
       [-0.29951677,  0.75988955,  0.18280267, -1.55023271,  0.33821802,
         0.36324148, -2.10052547, -0.4380675 , -

We'll fit a [Support Vector Classifier](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html), using [grid search](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to find the best value of the $C$ hyperparameter.

In [4]:
import numpy as np

In [30]:
C = np.logspace(0.001, 10, 40)

In [31]:
C

array([1.00230524e+00, 1.80877529e+00, 3.26414341e+00, 5.89052287e+00,
       1.06301272e+01, 1.91832893e+01, 3.46184558e+01, 6.24729922e+01,
       1.12739713e+02, 2.03451801e+02, 3.67152216e+02, 6.62568476e+02,
       1.19568116e+03, 2.15774441e+03, 3.89389840e+03, 7.02698831e+03,
       1.26810100e+04, 2.28843437e+04, 4.12974349e+04, 7.45259794e+04,
       1.34490717e+05, 2.42703994e+05, 4.37987321e+05, 7.90398585e+05,
       1.42636531e+06, 2.57404055e+06, 4.64515275e+06, 8.38271335e+06,
       1.51275721e+07, 2.72994468e+07, 4.92649971e+07, 8.89043635e+07,
       1.60438167e+08, 2.89529158e+08, 5.22488725e+08, 9.42891104e+08,
       1.70155564e+09, 3.07065320e+09, 5.54134749e+09, 1.00000000e+10])

In [32]:
param_grid = {"C": C,
              "kernel": ['rbf', 'poly', 'sigmoid'],
              "shrinking": [True, False]}

grid_search = GridSearchCV(SVC(gamma='auto', random_state=0, probability=True),
                           param_grid=param_grid,
                           return_train_score=False,
                           iid=True,
                           cv=3,
                           n_jobs=1)

To fit that normally, we would call

```python
grid_search.fit(X, y)
```

To fit it using the cluster, we just need to use a context manager provided by joblib.

In [33]:
%%time
grid_search.fit(X, y)

CPU times: user 40.8 s, sys: 27.5 ms, total: 40.8 s
Wall time: 40.8 s


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/model_selection/_search.py:849: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


GridSearchCV(cv=3,
             estimator=SVC(gamma='auto', probability=True, random_state=0),
             iid=True, n_jobs=1,
             param_grid={'C': array([1.00230524e+00, 1.80877529e+00, 3.26414341e+00, 5.89052287e+00,
       1.06301272e+01, 1.91832893e+01, 3.46184558e+01, 6.24729922e+01,
       1.12739713e+02, 2.03451801e+02, 3.67152216e+02, 6.62568476e+02,
       1.19568116e+03, 2.15774441e+03, 3.89389840e+03, 7.0269...
       1.34490717e+05, 2.42703994e+05, 4.37987321e+05, 7.90398585e+05,
       1.42636531e+06, 2.57404055e+06, 4.64515275e+06, 8.38271335e+06,
       1.51275721e+07, 2.72994468e+07, 4.92649971e+07, 8.89043635e+07,
       1.60438167e+08, 2.89529158e+08, 5.22488725e+08, 9.42891104e+08,
       1.70155564e+09, 3.07065320e+09, 5.54134749e+09, 1.00000000e+10]),
                         'kernel': ['rbf', 'poly', 'sigmoid'],
                         'shrinking': [True, False]})

In [18]:
cluster, client = init_cluster()

In [19]:
cluster

In [34]:
param_grid = {"C": C,
              "kernel": ['rbf', 'poly', 'sigmoid'],
              "shrinking": [True, False]}

grid_search = GridSearchCV(SVC(gamma='auto', random_state=0, probability=True),
                           param_grid=param_grid,
                           return_train_score=False,
                           iid=True,
                           cv=3,
                           n_jobs=-1)

In [35]:
import joblib

In [36]:
%%time
with joblib.parallel_backend('dask'):
    grid_search.fit(X, y)

CPU times: user 1.47 s, sys: 107 ms, total: 1.57 s
Wall time: 2.56 s


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/model_selection/_search.py:849: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


We fit 48 different models, one for each hyper-parameter combination in `param_grid`, distributed across the cluster. At this point, we have a regular scikit-learn model, which can be used for prediction, scoring, etc.

In [ ]:
pd.DataFrame(grid_search.cv_results_).head()

In [ ]:
grid_search.predict(X)[:5]

In [ ]:
grid_search.score(X, y)

For more on training scikit-learn models with distributed joblib, see the [dask-ml documentation](http://dask-ml.readthedocs.io/en/latest/joblib.html).

## Training on Large Datasets

Most estimators in scikit-learn are designed to work on in-memory arrays. Training with larger datasets may require different algorithms.

All of the algorithms implemented in Dask-ML work well on larger than memory datasets, which you might store in a [dask array](http://dask.pydata.org/en/latest/array.html) or [dataframe](http://dask.pydata.org/en/latest/dataframe.html).

In [ ]:
%matplotlib inline

In [ ]:
import dask_ml.datasets
import dask_ml.cluster
import matplotlib.pyplot as plt

In this example, we'll use `dask_ml.datasets.make_blobs` to generate some random *dask* arrays.

In [ ]:
X, y = dask_ml.datasets.make_blobs(n_samples=10000000,
                                   chunks=1000000,
                                   random_state=0,
                                   centers=3)
X = X.persist()
X

We'll use the k-means implemented in Dask-ML to cluster the points. It uses the `k-means||` (read: "k-means parallel") initialization algorithm, which scales better than `k-means++`. All of the computation, both during and after initialization, can be done in parallel.

In [ ]:
km = dask_ml.cluster.KMeans(n_clusters=3, init_max_iter=2, oversampling_factor=10)
km.fit(X)

We'll plot a sample of points, colored by the cluster each falls into.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X[::10000, 0], X[::10000, 1], marker='.', c=km.labels_[::10000],
           cmap='viridis', alpha=0.25);

For all the estimators implemented in Dask-ML, see the [API documentation](http://dask-ml.readthedocs.io/en/latest/modules/api.html).